# 1. Mô tả

Ở notebook này chúng ta sẽ có hai bước chính:

1. Tiền xử lý sơ bộ: Dựa vào insights rút ra từ notebook 1
    - Loại bỏ 2 cột Naive bayes
    - Encode các biến phân loại có thứ tự
    - Giữ lại top 10 cột tương quan cao tìm thấy ở notebook 1

2. Tạo giả thuyết và chuẩn bị tập train, valid, test cho bước modeling.

## Import thư viện

In [5]:
# import các thư viện cần thiết

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

# Cấu hình hiển thị
sns.set_theme(style="whitegrid")
warnings.filterwarnings("ignore")

## Khai báo hàm

In [11]:
# khai báo các hàm sẽ sử dụng (có thể bắt chước, copy lại hàm từ notebook1)
def clean_string_columns(data: np.ndarray) -> np.ndarray:
    """
    Làm sạch các cột dạng chuỗi trong structured array.

    Input:
        data (np.ndarray): structured array đọc từ np.genfromtxt
    Output:
        np.ndarray: structured array sau khi đã strip khoảng trắng và dấu nháy (", ')
    Nhiệm vụ:
        Do dữ liệu categorical đôi khi có dạng '"Blue"', '"M"', ... nên cần bỏ dấu nháy để xử lý đúng.
    """
    if data is None:
        return None

    for name in data.dtype.names:
        kind = data.dtype[name].kind  # 'U' (unicode), 'S' (bytes), 'O' (object), ...
        if kind in ("U", "S", "O"):
            col = data[name]
            if kind == "S":
                col = np.char.decode(col, "utf-8", errors="ignore")
            data[name] = np.char.strip(col.astype(str), chars=" \"'")

    return data


def load_data_numpy(filepath: str, clean_strings: bool = True) -> np.ndarray:
    """
    Đọc dữ liệu CSV bằng NumPy (structured array).

    Input:
        filepath (str): đường dẫn tới file CSV
        clean_strings (bool): nếu True thì làm sạch các cột dạng chuỗi sau khi load
    Output:
        np.ndarray: structured array chứa dữ liệu + tên cột (names=True)
    """
    if not os.path.exists(filepath):
        print(f"Lỗi: Không tìm thấy file tại đường dẫn: {filepath}")
        return None

    try:
        data = np.genfromtxt(filepath, delimiter=",", dtype=None, names=True, encoding="utf-8")
        if clean_strings:
            data = clean_string_columns(data)
        print("Đã load dữ liệu thành công!")
        return data
    except Exception as e:
        print(f"Có lỗi xảy ra khi đọc file: {e}")
        return None


def drop_columns_structured(data: np.ndarray, drop_cols: list) -> np.ndarray:
    """
    Loại bỏ một danh sách cột khỏi structured array.

    Input:
        data (np.ndarray): structured array
        drop_cols (list): danh sách tên cột cần loại
    Output:
        np.ndarray: structured array chỉ còn các cột giữ lại
    """
    if data is None:
        return None

    cols = list(data.dtype.names)
    keep_cols = [c for c in cols if c not in set(drop_cols)]
    return data[keep_cols]


def drop_naive_bayes_columns(data: np.ndarray) -> np.ndarray:
    """
    Loại bỏ 2 cột Naive_Bayes_Classifier_* (rác) khỏi dataset.

    Input:
        data (np.ndarray): structured array
    Output:
        np.ndarray: structured array đã loại các cột Naive_Bayes
    """
    if data is None:
        return None

    cols = list(data.dtype.names)
    drop_cols = [c for c in cols if "Naive_Bayes_Classifier" in c]

    if len(drop_cols) == 0:
        print("Không thấy cột Naive_Bayes_Classifier nào để loại.")
        return data

    print("Các cột bị loại:")
    for c in drop_cols:
        print("-", c)

    return drop_columns_structured(data, drop_cols)


def encode_ordinal(col: np.ndarray, ordered_values: list, unknown_value: float = np.nan) -> np.ndarray:
    """
    Mã hoá một cột categorical theo thứ tự (ordinal encoding).

    Input:
        col (np.ndarray): mảng 1 chiều của cột categorical
        ordered_values (list): danh sách mức theo thứ tự (mức 0 -> mức 1 -> ...)
        unknown_value (float): giá trị gán cho các mức không nằm trong ordered_values (mặc định NaN)
    Output:
        np.ndarray: mảng float 1 chiều đã được encode (0..k-1) hoặc unknown_value
    """
    col = np.char.strip(col.astype(str), chars=" \"'")
    encoded = np.full(col.shape[0], unknown_value, dtype=float)

    for i, v in enumerate(ordered_values):
        encoded[col == v] = float(i)

    return encoded


def encode_binary(col: np.ndarray, mapping: dict, unknown_value: float = np.nan) -> np.ndarray:
    """
    Mã hoá một cột categorical theo mapping 2 lớp (hoặc vài lớp nhỏ), ví dụ Gender: F->0, M->1.

    Input:
        col (np.ndarray): mảng 1 chiều (str/bytes)
        mapping (dict): ví dụ {'F': 0.0, 'M': 1.0}
        unknown_value (float): giá trị cho các case không có trong mapping
    Output:
        np.ndarray: mảng float 1 chiều
    """
    col = np.char.strip(col.astype(str), chars=" \"'")
    encoded = np.full(col.shape[0], unknown_value, dtype=float)

    for k, v in mapping.items():
        encoded[col == k] = float(v)

    return encoded


def build_feature_matrix_for_corr(data: np.ndarray) -> tuple[np.ndarray, list]:
    """
    Tạo ma trận X để tính tương quan Pearson (có encode categorical), và danh sách feature_names tương ứng.

    Thành phần:
        - Cột 0: Attrition_Flag đã mã hoá 0/1
        - Các cột tiếp theo: categorical đã encode (Gender + ordinal cho 4 cột còn lại)
        - Các cột số: giữ nguyên (loại CLIENTNUM)

    Input:
        data (np.ndarray): structured array đã được clean string
    Output:
        X (np.ndarray): shape (n_samples_valid, n_features), dtype float
        feature_names (list): tên cột tương ứng với X

    Ghi chú:
        - Unknown không nên xem là “mức cao nhất”, nên ở đây ta KHÔNG đưa 'Unknown' vào ordered_values,
          để nó rơi vào unknown_value=np.nan rồi lọc dòng trước khi tính corr.
    """
    # target
    flag = np.char.strip(data["Attrition_Flag"].astype(str), chars=" \"'")
    y = np.where(flag == "Attrited Customer", 1.0, 0.0)

    # numeric
    numeric_cols = [n for n in data.dtype.names if np.issubdtype(data.dtype[n], np.number)]
    if "CLIENTNUM" in numeric_cols:
        numeric_cols.remove("CLIENTNUM")
    X_num = np.column_stack([data[c].astype(float) for c in numeric_cols]) if numeric_cols else np.empty((data.shape[0], 0))

    # categorical enc
    gender = encode_binary(data["Gender"], {"F": 0.0, "M": 1.0}, unknown_value=np.nan)

    edu = encode_ordinal(
        data["Education_Level"],
        ["Uneducated", "High School", "College", "Graduate", "Post-Graduate", "Doctorate"],
        unknown_value=np.nan
    )
    income = encode_ordinal(
        data["Income_Category"],
        ["Less than $40K", "$40K - $60K", "$60K - $80K", "$80K - $120K", "$120K +"],
        unknown_value=np.nan
    )
    card = encode_ordinal(
        data["Card_Category"],
        ["Blue", "Silver", "Gold", "Platinum"],
        unknown_value=np.nan
    )
    marital = encode_ordinal(
        data["Marital_Status"],
        ["Single", "Married", "Divorced"],
        unknown_value=np.nan
    )

    X_cat = np.column_stack([gender, edu, income, card, marital])
    cat_names = ["Gender_enc", "Education_Level_enc", "Income_Category_enc", "Card_Category_enc", "Marital_Status_enc"]

    X = np.column_stack([y, X_cat, X_num])
    feature_names = ["Attrition_Flag"] + cat_names + numeric_cols

    # lọc các dòng có NaN/inf để corr không bị NaN hàng loạt
    valid_rows = np.all(np.isfinite(X), axis=1)
    X = X[valid_rows]
    return X, feature_names


def get_top_k_features_from_corr(X: np.ndarray, feature_names: list, k: int = 10) -> list:
    """
    Lấy top-k feature có |corr| lớn nhất với target (cột Attrition_Flag nằm ở index 0).

    Input:
        X (np.ndarray): ma trận (n_samples, n_features), cột 0 là target
        feature_names (list): danh sách tên feature tương ứng với cột X
        k (int): số lượng feature cần lấy
    Output:
        list: danh sách tên top-k features (không bao gồm target)
    """
    corr = np.corrcoef(X, rowvar=False)
    corr_with_target = corr[0, 1:]  # bỏ target
    abs_corr = np.abs(corr_with_target)

    other_features = np.array(feature_names[1:])
    top_idx = np.argsort(abs_corr)[::-1][:k]
    return other_features[top_idx].tolist()

In [9]:
def make_preprocessed_topk_structured(data: np.ndarray, top_features: list) -> np.ndarray:
    """
    Tạo dataset đã tiền xử lý dạng structured array float:
    - Attrition_Flag -> 0/1
    - Chỉ giữ các cột trong top_features (đã encode nếu là categorical *_enc)
    - Lọc các dòng có NaN/inf (do Unknown -> NaN) để dữ liệu sạch khi train.
    """
    if data is None:
        return None

    # target
    flag = np.char.strip(data["Attrition_Flag"].astype(str), chars=" \\\"'")
    y = np.where(flag == "Attrited Customer", 1.0, 0.0)

    # tái tạo đúng encoding như lúc tính corr
    gender = encode_binary(data["Gender"], {"F": 0.0, "M": 1.0}, unknown_value=np.nan)
    edu = encode_ordinal(
        data["Education_Level"],
        ["Uneducated", "High School", "College", "Graduate", "Post-Graduate", "Doctorate"],
        unknown_value=np.nan
    )
    income = encode_ordinal(
        data["Income_Category"],
        ["Less than $40K", "$40K - $60K", "$60K - $80K", "$80K - $120K", "$120K +"],
        unknown_value=np.nan
    )
    card = encode_ordinal(
        data["Card_Category"],
        ["Blue", "Silver", "Gold", "Platinum"],
        unknown_value=np.nan
    )
    marital = encode_ordinal(
        data["Marital_Status"],
        ["Single", "Married", "Divorced"],
        unknown_value=np.nan
    )

    encoded_map = {
        "Gender_enc": gender,
        "Education_Level_enc": edu,
        "Income_Category_enc": income,
        "Card_Category_enc": card,
        "Marital_Status_enc": marital,
    }

    feature_cols = []
    final_names = ["Attrition_Flag"]

    for f in top_features:
        if f in encoded_map:
            feature_cols.append(encoded_map[f].astype(float))
        else:
            feature_cols.append(data[f].astype(float))
        final_names.append(f)

    M = np.column_stack([y] + feature_cols).astype(float)

    # lọc dòng không hợp lệ
    valid_rows = np.all(np.isfinite(M), axis=1)
    M = M[valid_rows]

    # build structured array float
    dtype = [(name, "f8") for name in final_names]
    out = np.empty(M.shape[0], dtype=dtype)
    for j, name in enumerate(final_names):
        out[name] = M[:, j]

    return out


def save_float_structured_csv(data: np.ndarray, out_path: str) -> None:
    """Lưu structured array (toàn float) ra CSV (có header)."""
    if data is None:
        print("Không có dữ liệu để lưu.")
        return

    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    names = list(data.dtype.names)
    arr2d = np.column_stack([data[n] for n in names]).astype(float)
    header = ",".join(names)

    np.savetxt(out_path, arr2d, delimiter=",", header=header, comments="", fmt="%.6g")
    print("Đã lưu file:", out_path)

In [25]:
def numpy_train_valid_test_split(X, y, valid_size=0.1, test_size=0.1, random_state=42):
    """
    Chia dữ liệu thành 3 tập: Train, Validation, Test sử dụng Numpy.
    
    Input:
        X: np.ndarray - Ma trận đặc trưng
        y: np.ndarray - Biến mục tiêu
        valid_size: float - Tỷ lệ tập Validation (Mặc định 0.1)
        test_size: float - Tỷ lệ tập Test (Mặc định 0.1)
        random_state: int - Seed ngẫu nhiên
        
    Output:
        (X_train, y_train), (X_val, y_val), (X_test, y_test)
    """
    np.random.seed(random_state)
    n_samples = X.shape[0]
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    # Tính số lượng mẫu
    n_valid = int(n_samples * valid_size)
    n_test = int(n_samples * test_size)
    n_train = n_samples - n_valid - n_test
    
    # Cắt index
    train_idx = indices[:n_train]
    val_idx = indices[n_train : n_train + n_valid]
    test_idx = indices[n_train + n_valid:]
    
    return (X[train_idx], y[train_idx]), (X[val_idx], y[val_idx]), (X[test_idx], y[test_idx])

def save_modeling_data(filename, X, y, header):
    """
    Lưu dữ liệu modeling xuống file CSV tại folder ../data/modeling/
    """
    # Tạo folder nếu chưa có
    MODELING_PATH = '../data/modeling'
    if not os.path.exists(MODELING_PATH):
        os.makedirs(MODELING_PATH)
        
    # Reshape y để thành cột dọc (n, 1) nếu cần
    if len(y.shape) == 1:
        y = y.reshape(-1, 1)
        
    data_to_save = np.hstack((X, y))
    filepath = os.path.join(MODELING_PATH, filename)
    
    # Lưu bằng numpy
    np.savetxt(filepath, data_to_save, delimiter=",", header=header, comments='', fmt='%f')
    print(f"Đã lưu file: {filepath} | Shape: {data_to_save.shape}")

## Load dữ liệu

In [14]:
data_np = load_data_numpy('../data/raw/original_data.csv', clean_strings=True)

Đã load dữ liệu thành công!


#  2. Tiền xử lý sơ bộ

In [15]:
# 1) Drop Naive Bayes
data_np = drop_naive_bayes_columns(data_np)

# 2) Tính corr sau khi encode (để lấy top 10)
X_all, feature_names = build_feature_matrix_for_corr(data_np)
top10_features = get_top_k_features_from_corr(X_all, feature_names, k=10)

print("Top 10 features (sau encode) theo |corr| với Attrition_Flag:")
corr = np.corrcoef(X_all, rowvar=False)
for f in top10_features:
    idx = feature_names.index(f)
    print(f"- {f}: corr = {corr[0, idx]:.4f}")

# 3) Tạo data_np mới: chỉ gồm target + top10 (đã encode) rồi gán lại
data_np = make_preprocessed_topk_structured(data_np, top10_features)

print("\nDataset sau preprocessing:")
print("dtype names:", data_np.dtype.names)
print("shape:", data_np.shape)
print("churn rate (mean y):", data_np["Attrition_Flag"].mean())

# 4) Lưu ra data/processed/
out_csv = os.path.join("..", "data", "processed", "preprocessed_data.csv")
save_float_structured_csv(data_np, out_csv)

Các cột bị loại:
- Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1
- Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
Top 10 features (sau encode) theo |corr| với Attrition_Flag:
- Total_Trans_Ct: corr = -0.3573
- Total_Ct_Chng_Q4_Q1: corr = -0.2805
- Total_Revolving_Bal: corr = -0.2654
- Contacts_Count_12_mon: corr = 0.1939
- Avg_Utilization_Ratio: corr = -0.1840
- Total_Trans_Amt: corr = -0.1604
- Months_Inactive_12_mon: corr = 0.1527
- Total_Relationship_Count: corr = -0.1457
- Total_Amt_Chng_Q4_Q1: corr = -0.1327
- Gender_enc: corr = -0.0354

Dataset sau preprocessing:
dtype names: ('Attrition_Flag', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Total_Revolving_Bal', 'Contacts_Count_12_mon', 'Avg_Utilization_Ratio', 'Total_Trans_Amt', 'Months_Inactive_12_mon', 'Total_Relationship_Count', 'Total_Amt_Chng_Q4_Q1', 'Gender_enc'

In [19]:
# đọc lại file processed vào biến mới để tiện re run code
# để đảm bảo lưu đúng kiểu float các cột số, chúng ta dùng genfromtxt
file_processed = '../data/processed/preprocessed_data.csv'

processed_np = np.genfromtxt(
    file_processed,
    delimiter=",",
    names=True,
    dtype=float,
    encoding="utf-8"
)

print("Đã load processed_np!")
print("shape:", processed_np.shape)
print("columns:", processed_np.dtype.names)

Đã load processed_np!
shape: (10127,)
columns: ('Attrition_Flag', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Total_Revolving_Bal', 'Contacts_Count_12_mon', 'Avg_Utilization_Ratio', 'Total_Trans_Amt', 'Months_Inactive_12_mon', 'Total_Relationship_Count', 'Total_Amt_Chng_Q4_Q1', 'Gender_enc')


In [20]:
processed_np[:5]

array([(0., 42., 1.625,  777., 3., 0.061, 1144., 1., 5., 1.335, 1.),
       (0., 33., 3.714,  864., 2., 0.105, 1291., 1., 6., 1.541, 0.),
       (0., 20., 2.333,    0., 0., 0.   , 1887., 1., 4., 2.594, 1.),
       (0., 20., 2.333, 2517., 1., 0.76 , 1171., 4., 3., 1.405, 0.),
       (0., 28., 2.5  ,    0., 0., 0.   ,  816., 1., 5., 2.175, 1.)],
      dtype=[('Attrition_Flag', '<f8'), ('Total_Trans_Ct', '<f8'), ('Total_Ct_Chng_Q4_Q1', '<f8'), ('Total_Revolving_Bal', '<f8'), ('Contacts_Count_12_mon', '<f8'), ('Avg_Utilization_Ratio', '<f8'), ('Total_Trans_Amt', '<f8'), ('Months_Inactive_12_mon', '<f8'), ('Total_Relationship_Count', '<f8'), ('Total_Amt_Chng_Q4_Q1', '<f8'), ('Gender_enc', '<f8')])

# 3. Tạo giả thuyết

Ở bước này chúng ta sẽ đến với việc tạo và thử nghiệm các giả thuyết. Vì lý do hạn chế khả năng và thời gian, em chỉ có thể đưa ra hai giả thuyết. 

Mô hình sẽ sử dụng là Logistic Regression vì dễ cài đặt bằng Numpy, dễ hiểu (đã được học).

## 3.1. Giả thuyết 1

### Mô hình Hoạt động Tuyến tính

**Mô tả giả thuyết:**
Đây là mô hình cơ sở (Baseline). Giả thuyết cho rằng quyết định rời bỏ là kết quả của phép cộng dồn tuyến tính giữa các chỉ số hoạt động quan trọng nhất. Các biến số tác động độc lập với nhau, không có sự tương tác chéo.

Mô hình sẽ sử dụng 3 biến đặc trưng có tương quan mạnh nhất (theo kết quả EDA):
1.  `Total_Trans_Ct` (Tổng số lần giao dịch) - Tương quan Âm.
2.  `Total_Revolving_Bal` (Tổng dư nợ xoay vòng) - Tương quan Âm.
3.  `Contacts_Count_12_mon` (Số lần liên hệ) - Tương quan Dương.

**Cơ sở thực tế (Rationale):**
* **Tính đơn giản:** Dựa trên tư duy thông thường: Khách hàng càng giao dịch nhiều + càng nợ nhiều + càng ít phàn nàn => Càng gắn bó.
* **Đại diện đa chiều:** 3 biến này đại diện cho 3 khía cạnh khác nhau: Hành vi tiêu dùng, Tình trạng tài chính và Sự hài lòng.

**Công thức Logistic (Linear):**
$$P(Y=1|X) = \sigma(w_0 + w_1 \cdot \text{Trans\_Ct} + w_2 \cdot \text{Rev\_Bal} + w_3 \cdot \text{Contacts})$$

### Chia tập train, valid, test 

In [26]:
# 1. Load dữ liệu
data_path = '../data/processed/preprocessed_data.csv'
with open(data_path, 'r') as f:
    headers = f.readline().strip().split(',')

# Map index
idx_target = headers.index('Attrition_Flag')
idx_ct = headers.index('Total_Trans_Ct')
idx_bal = headers.index('Total_Revolving_Bal')
idx_contact = headers.index('Contacts_Count_12_mon')

data_full = np.genfromtxt(data_path, delimiter=',', skip_header=1)

# 2. Chọn Features
feat_ct = data_full[:, idx_ct]
feat_bal = data_full[:, idx_bal]
feat_contact = data_full[:, idx_contact]
target = data_full[:, idx_target]

# Gom lại thành ma trận X
X_h1 = np.column_stack((feat_ct, feat_bal, feat_contact))
y_h1 = target

print(f"H1 Dataset shape: {X_h1.shape}")

# 3. Chia tập Train/Valid/Test (80/10/10)
(X_train, y_train), (X_val, y_val), (X_test, y_test) = numpy_train_valid_test_split(
    X_h1, y_h1, valid_size=0.1, test_size=0.1
)

# 4. Lưu file modeling
header_h1 = "Total_Trans_Ct,Total_Revolving_Bal,Contacts_Count_12_mon,Attrition_Flag"
save_modeling_data('h1_train.csv', X_train, y_train, header_h1)
save_modeling_data('h1_valid.csv', X_val, y_val, header_h1)
save_modeling_data('h1_test.csv', X_test, y_test, header_h1)

H1 Dataset shape: (10127, 3)
Đã lưu file: ../data/modeling\h1_train.csv | Shape: (8103, 4)
Đã lưu file: ../data/modeling\h1_valid.csv | Shape: (1012, 4)
Đã lưu file: ../data/modeling\h1_test.csv | Shape: (1012, 4)


## 3.2. Giả thuyết 2

### Interaction "Dead Zone" Model (Mô hình Tương tác "Vùng Tử Thần")

**Mô tả giả thuyết:**
Giả thuyết này cho rằng rủi ro rời bỏ đến từ sự cộng hưởng tiêu cực giữa **Tần suất** và **Giá trị chi tiêu**.

Mô hình sử dụng:
1.  Các biến gốc: `Total_Trans_Ct`, `Total_Trans_Amt`, `Total_Revolving_Bal`.
2.  **Biến tương tác:** `Engagement_Score = Total_Trans_Ct * Total_Trans_Amt`.

**Công thức Logistic (Interaction):**
$$P(Y=1|X) = \sigma(w_0 + w_1 \cdot Ct + w_2 \cdot Amt + w_3 \cdot Bal + w_4 \cdot (Ct \times Amt))$$

In [28]:
# 1. Load dữ liệu (để H2 chạy độc lập, không phụ thuộc H1)
data_path = '../data/processed/preprocessed_data.csv'
with open(data_path, 'r') as f:
    headers = f.readline().strip().split(',')

data_full = np.genfromtxt(data_path, delimiter=',', skip_header=1)

# 2. Map index
idx_target = headers.index('Attrition_Flag')
idx_ct = headers.index('Total_Trans_Ct')
idx_amt = headers.index('Total_Trans_Amt')
idx_bal = headers.index('Total_Revolving_Bal')

# 3. Feature Engineering
feat_ct = data_full[:, idx_ct]
feat_amt = data_full[:, idx_amt]
feat_bal = data_full[:, idx_bal]
target = data_full[:, idx_target]

# Biến tương tác (đúng theo mô tả: Engagement_Score = Ct * Amt)
feat_engagement = feat_ct * feat_amt

# Gom lại thành ma trận X
X_h2 = np.column_stack((feat_ct, feat_amt, feat_bal, feat_engagement))
y_h2 = target

print(f"H2 Dataset shape: {X_h2.shape}")

# 4. Chia tập Train/Valid/Test (80/10/10)
(X_train_h2, y_train_h2), (X_val_h2, y_val_h2), (X_test_h2, y_test_h2) = numpy_train_valid_test_split(
    X_h2, y_h2, valid_size=0.1, test_size=0.1
)

# 5. Lưu file modeling
header_h2 = "Total_Trans_Ct,Total_Trans_Amt,Total_Revolving_Bal,Engagement_Score,Attrition_Flag"
save_modeling_data('h2_train.csv', X_train_h2, y_train_h2, header_h2)
save_modeling_data('h2_valid.csv', X_val_h2, y_val_h2, header_h2)
save_modeling_data('h2_test.csv', X_test_h2, y_test_h2, header_h2)

H2 Dataset shape: (10127, 4)
Đã lưu file: ../data/modeling\h2_train.csv | Shape: (8103, 5)
Đã lưu file: ../data/modeling\h2_valid.csv | Shape: (1012, 5)
Đã lưu file: ../data/modeling\h2_test.csv | Shape: (1012, 5)
